# Encryption as a Service using MySQL
Based on
https://github.com/assareh/transit-app-example

This scenario demonstrates dynamically generated MySQL credentials, Encryption as a Service and Format Preserving Encryption all together.

## Prerequisites

### Set Up Environment Variables

Customize the values below if you want Vault to start with different parameters

In [ ]:
export RED="\e[0;31m" YELLOW="\e[0;33m" BLDYELLOW="\e[1;33m" GREEN="\e[0;32m"
export CYAN="\e[0;36m" BLUE="\e[0;34m" WHITE="\e[0;37m" BLDWHITE="\e[1;37m"
export NC="\e[0m"
# Some commands may have sensitive information. Prevent commands starting with a space to be saved to shell history.
HISTCONTROL=ignoreboth # do not save lines that begin with space in history
# Set dir env vars. 
# MAIN_DIR=$(pwd)
# WORK_DIR=config/vault

In [ ]:
# Common
export VAULT_VER=1.7.5 # 1.8+ enterprise requires license file
export VAULT_PORT=8200
export VAULT_TOKEN=root
export VAULT_ADDR=http://localhost:${VAULT_PORT}
export VAULT_LICENSE=$(cat ../../license/vault.hclic)
export LOGS_PATH=$(PWD)/vault_logs

In [ ]:
# Create required directories.
mkdir -p $WORK_DIR/{data,logs,config}

In [ ]:
## Create docker-compose file.
cat > docker-compose.yaml << EOF
version: '3.8'
services:
  vault:
    image: hashicorp/vault-enterprise:${VAULT_VER}_ent # Vault Enterprise
    #image: hashicorp/vault:${VAULT_VER}                # Vault OSS
    container_name: vault
    restart: always
    volumes:
      - ./${WORK_DIR}/logs:/vault/logs
    ports:
      - "8200:8200/tcp"
    environment:
      VAULT_DEV_ROOT_TOKEN_ID: ${VAULT_TOKEN}
      VAULT_DEV_LISTEN_ADDRESS: "0.0.0.0:${VAULT_PORT}"
      VAULT_ADDR: ${VAULT_ADDR}
      VAULT_LICENSE: ${VAULT_LICENSE}
    cap_add:
      - IPC_LOCK
    entrypoint: "vault server -dev" # dev mode
EOF

In [ ]:
docker compose up -d --quiet-pull > docker-compose.log 2>&1

In [ ]:
vault status

### Get Credentials

In [ ]:
export creds=$(vault token create -format=json -period=30m | jq .auth.client_token -r)
echo $creds

## Create a database

In [ ]:
docker run --name workshop-mysql \
  -p 3306:3306 \
  --rm \
  -e MYSQL_ROOT_PASSWORD=root \
  -e MYSQL_ROOT_HOST=% \
  -e MYSQL_DATABASE=my_app \
  -e MYSQL_USER=vault \
  -e MYSQL_PASSWORD=vaultpw \
  -d mysql/mysql-server:5.7

### Verify Database functionality

Display existing database users

In [ ]:
docker exec -it workshop-mysql mysql -uroot -proot -e "select user from mysql.user;"

Display existing databases

In [ ]:
docker exec -it workshop-mysql mysql -uroot -proot -e "show databases"

## Configure Vault and Database Engine

In [ ]:
vault audit enable file file_path=/tmp/vault_audit.log

Enable Database secrets engine.

In [ ]:
vault secrets enable -path=lob_a/workshop/database database

Configure database connection.

In [ ]:
vault write lob_a/workshop/database/config/ws-mysql-database \
    plugin_name=mysql-database-plugin \
    connection_url="{{username}}:{{password}}@tcp(host.docker.internal:3306)/" \
    allowed_roles="workshop-app-long,workshop-app" \
    username="root" \
    password="root"

Configure Vault `workshop-app` role

In [ ]:
vault write lob_a/workshop/database/roles/workshop-app-long \
    db_name=ws-mysql-database \
    creation_statements="CREATE USER '{{name}}'@'%' IDENTIFIED BY '{{password}}';GRANT ALL ON *.* TO '{{name}}'@'%';" \
    default_ttl="1h" \
    max_ttl="24h"

In [ ]:
vault write lob_a/workshop/database/roles/workshop-app \
    db_name=ws-mysql-database \
    creation_statements="CREATE USER '{{name}}'@'%' IDENTIFIED BY '{{password}}';GRANT ALL ON *.* TO '{{name}}'@'%';" \
    default_ttl="5m" \
    max_ttl="1h"

## Demo

### Get Database credential

In [ ]:
vault read lob_a/workshop/database/creds/workshop-app-long

Display database users. Note new user added by Vault

In [ ]:
docker exec -it workshop-mysql mysql -u root -p'root' \
  -e "select user from mysql.user"; 

### Configure Encryption as a Service 

Enable Transit secrets engine.

In [ ]:
vault secrets enable -path=lob_a/workshop/transit transit

In [ ]:

vault write -f lob_a/workshop/transit/keys/customer-key

In [ ]:
vault write -f lob_a/workshop/transit/keys/archive-key

### Configure Transform

In [ ]:
TRANSFORM_PATH=lob_a/workshop/transform
vault secrets enable -path=${TRANSFORM_PATH} transform

In [ ]:
vault write ${TRANSFORM_PATH}/role/ssn transformations=ssn-fpe

In [ ]:
vault write ${TRANSFORM_PATH}/transformation/ssn-fpe \
  type=fpe \
  template=builtin/socialsecuritynumber \
  tweak_source=internal \
  allowed_roles=ssn

## RUN the transit app 

**Note:** after a while the transit-app-example gives 500 Internal Server Error.  Just stop and rerun the container.

Clone Andy Assareh's repo.

In [ ]:
git clone https://github.com/assareh/transit-app-example.git

Build transit app container

In [ ]:
pushd transit-app-example/backend

In [ ]:
docker build -q -t transit-app .

In [ ]:
popd

Run the transit app container

In [ ]:
docker run --name transit-app \
  --rm \
  -p 5000:5000 \
  -e VAULT_ADDR=http://host.docker.internal:8200 \
  -e VAULT_DATABASE_CREDS_PATH=lob_a/workshop/database/creds/workshop-app-long \
  -e VAULT_NAMESPACE=root \
  -e VAULT_TOKEN=${creds} \
  -e VAULT_TRANSFORM_PATH=${TRANSFORM_PATH} \
  -e VAULT_TRANSFORM_MASKING_PATH=lob_a/workshop/transform \
  -e VAULT_TRANSIT_PATH=lob_a/workshop/transit \
  -e MYSQL_ADDR=host.docker.internal \
  -d transit-app

* `VAULT_ADDR`
    * `host.docker.internal` - special DNS name which resolves to internal IP used by host. Mac only.
    
* `VAULT_TRANSFORM_PATH` and `VAULT_TRANSFORM_MASKING_PATH` should not be the same?

See the additional database accounts created.

In [ ]:
docker exec -it workshop-mysql mysql -uroot -proot -e  "select user from mysql.user;"

Open the transit app in a browser.

In [ ]:
open http://localhost:5000

## Cleanup

Stop transit app

In [ ]:
docker stop transit-app

Stop mysql database container

In [ ]:
docker stop workshop-mysql

Stop Vault container

In [ ]:
docker compose down -h

Delete artifacts

In [ ]:
rm -rf transit-app-example

## End